In [230]:
import random
from copy import deepcopy
import math

In [231]:
service_time = 10 # trebalo bi promeniti kroz argument/parametar pri prevodjenju programa

def load(file):
    cities = []
    with open(file, 'r') as f:
        for line in f:
            data = line.split()
            city_num = int(data[0])-1
            x = float(data[1])
            y = float(data[2])
            demand = float(data[3])
            ready_time = float(data[4])
            due_time = float(data[5])
            
            city = {'city_num' : city_num, 'x' : x, 'y' : y, 'demand' : demand, 'ready_time' : ready_time, 'due_time' : due_time}
            cities.append(city)
            
    return cities   

In [232]:
cities = load('input.txt')
for city in cities:
    print(city)

{'city_num': 0, 'x': 35.0, 'y': 35.0, 'demand': 0.0, 'ready_time': 0.0, 'due_time': 230.0}
{'city_num': 1, 'x': 41.0, 'y': 49.0, 'demand': 10.0, 'ready_time': 161.0, 'due_time': 171.0}
{'city_num': 2, 'x': 35.0, 'y': 17.0, 'demand': 7.0, 'ready_time': 50.0, 'due_time': 60.0}
{'city_num': 3, 'x': 55.0, 'y': 45.0, 'demand': 13.0, 'ready_time': 116.0, 'due_time': 126.0}
{'city_num': 4, 'x': 55.0, 'y': 20.0, 'demand': 19.0, 'ready_time': 149.0, 'due_time': 159.0}
{'city_num': 5, 'x': 15.0, 'y': 30.0, 'demand': 26.0, 'ready_time': 34.0, 'due_time': 44.0}
{'city_num': 6, 'x': 25.0, 'y': 30.0, 'demand': 3.0, 'ready_time': 99.0, 'due_time': 109.0}
{'city_num': 7, 'x': 20.0, 'y': 50.0, 'demand': 5.0, 'ready_time': 81.0, 'due_time': 91.0}
{'city_num': 8, 'x': 10.0, 'y': 43.0, 'demand': 9.0, 'ready_time': 95.0, 'due_time': 105.0}
{'city_num': 9, 'x': 55.0, 'y': 60.0, 'demand': 16.0, 'ready_time': 97.0, 'due_time': 107.0}
{'city_num': 10, 'x': 30.0, 'y': 60.0, 'demand': 16.0, 'ready_time': 124.0, 

In [233]:
class Individual:
    def __init__(self, data, capacity, num_of_vehicles, num_of_customers, service_time):
        self.num_of_vehicles = num_of_vehicles
        self.num_of_customers = num_of_customers
        self.depot = data[0]
        self.data = data[1:]
        self.capacity = capacity
        self.service_time = service_time

        self.solution = self.generate_solution()
        self. fitness = self.calc_fitness()

    def __len__(self) -> int:
        return len(self.solution)
    

    def calc_distance(self, customer1, customer2):
        return math.sqrt((customer1["x"] - customer2["x"])**2 + (customer1["y"] - customer2["y"])**2)

    def generate_solution(self):
        #kako kodirati hromozom? - moze npr u jednom nizu gde ide prvo redni broj vozila na kojise doda broj customera pa onda redni brojevi customera koje obilazi na svojoj ruti pa onda naredno vozilo itd
        #moraju da se provere svi uslovi, u zavisnosti da li radimo klasican vrp ili sa time windowom 
        # random inicijalizacija 
        
        #gradovi: depo = 0, preostali = 1...num_of_customers-1 
        #vozila = num_of_customers + i , i = 0..num_of_vehicle-1
        
        
        remaining = set(range(self.num_of_customers))
        code = []
        
        for i in range(self.num_of_vehicles):
            route = []
            capacity_remaining = self.capacity
            current_time = 0
            current_node = self.depot

            while remaining:

                feasible = [c for c in remaining if capacity_remaining >= self.data[c]["demand"] and current_time + self.calc_distance(current_node, self.data[c]) <= self.data[c]["due_time"]]
               

                if feasible: #ako nije prazan
                    customer_index = random.choice(feasible) # biramo random customera
                    customer = self.data[customer_index]
                    route.append(customer_index+1)
                    remaining.remove(customer_index)
                    capacity_remaining -= customer["demand"]
                    current_time += max(self.calc_distance(current_node, customer), customer["ready_time"] )+ self.service_time
                    current_node = customer

                else:
                    break
            if(len(route)>0):
                route.append(self.num_of_customers + i + 1)
            code.extend(route)

        code.pop(-1)
        return code

        
        
        

    def calc_fitness(self):
        # The value of travel time is equal to the value of distance. !!!!
        fitness = 0
        
        # kasnjenje, kapacitet, ????
        routes = [[] for _ in range(self.num_of_vehicles)]
        idx = 0
        for i in self.solution:
            if i > self.num_of_customers:
                idx+=1
            else:
                routes[idx].append(i-1)
        
        for route in routes:
            if len(route) != 0:
                current_time = self.calc_distance(self.depot, self.data[route[0]])
                for r,i in zip(route,range(1,len(route))):
                    if self.data[r]["due_time"] <= current_time:
                        return float('inf')
                    else:
                        current_time = max(current_time, self.data[r]["ready_time"]) + 1
                    current_time += self.calc_distance(self.data[r], self.data[route[i]])
                fitness += current_time + self.calc_distance(self.data[route[-1]], self.depot)

        return fitness

        # distance ~ isto sto i vreme 
        # vreme na putu + vreme cekanja (ako ga ima) + service time 
        # vreme cekanja = 0 ili ready time - arrival time

        # Di + distance(i,j) + wait time

        # penalizujemo resenja koja ne ispunjavaju uslove za time window


    def is_feasable(self):
        # ponavljanje gradova
        # koriscenje vise vozila nego sto postoji

        pass


   


In [234]:
x = Individual(cities, 200, 100, 100, 10)
y = Individual(cities, 200, 100, 100, 10)


In [235]:
x.solution


[35,
 58,
 101,
 60,
 89,
 102,
 79,
 56,
 103,
 38,
 91,
 104,
 66,
 70,
 105,
 40,
 100,
 106,
 27,
 20,
 107,
 71,
 80,
 108,
 99,
 10,
 109,
 4,
 25,
 110,
 18,
 93,
 111,
 75,
 73,
 112,
 82,
 96,
 113,
 98,
 55,
 114,
 33,
 77,
 115,
 68,
 24,
 116,
 83,
 85,
 117,
 67,
 54,
 118,
 59,
 94,
 17,
 119,
 64,
 97,
 120,
 12,
 48,
 121,
 11,
 49,
 122,
 39,
 6,
 123,
 41,
 43,
 124,
 52,
 74,
 125,
 69,
 87,
 126,
 5,
 8,
 127,
 86,
 37,
 128,
 44,
 1,
 129,
 9,
 34,
 130,
 3,
 13,
 131,
 14,
 22,
 132,
 50,
 133,
 21,
 26,
 134,
 76,
 32,
 135,
 63,
 62,
 136,
 61,
 84,
 137,
 36,
 7,
 138,
 57,
 139,
 51,
 140,
 90,
 141,
 23,
 142,
 81,
 143,
 45,
 78,
 144,
 95,
 15,
 145,
 88,
 46,
 146,
 30,
 147,
 65,
 148,
 16,
 149,
 53,
 150,
 2,
 151,
 31,
 19,
 152,
 92,
 42,
 153,
 29,
 154,
 47,
 155,
 72,
 156,
 28]

In [236]:
print(x.fitness)

5957.850035158913


In [237]:
def random_selection(population):
    return random.choice(population)

In [238]:
def tournament_selection(population, tournament_size):
    tournament = random.sample(population, tournament_size)
    return max(tournament, key = lambda x: x.fitness)

In [239]:
def roulette_selection(population):
    total_fitness = sum([individual.fitness for individual in population])
    selection_probs = [individual.fitness / total_fitness for individual in population]
    return random.choices(population, weights=selection_probs)[0]

In [240]:
def rang_selection(population):
    n = len(population)
    rank_sum = n * (n+1) / 2
    selection_probs = []
    for rank, _ in enumerate(sorted(population, key=lambda x: x.fitness), 1):
        selection_probs.append(rank / rank_sum)
        
    return random.choices(population, weights=selection_probs)[0]

In [241]:
class Example_Individual:
    def __init__(self, fitness, name):
        self.fitness = fitness
        self.name = name

example_population = [Example_Individual(fitness=0.8, name='A'), 
                      Example_Individual(fitness=0.6, name='B'), 
                      Example_Individual(fitness=0.7, name='C'),
                      Example_Individual(fitness=0.8, name='D'), 
                      Example_Individual(fitness=0.9, name='E')]

print(random_selection(example_population).name)
print(tournament_selection(example_population, 3).name)
print(roulette_selection(example_population).name)
print(rang_selection(example_population).name)

E
D
A
C


In [242]:
def order_crossover(parent1, parent2, child):
    lb, ub = sorted(random.sample(range(len(parent1) + 1), 2))
    
    child[lb:ub] = parent1[lb:ub]
        
    j = 0
    for i in range(len(child)):
        if j == lb:
            j = ub + 1
            
        if j == len(child):
            break
        
        if parent2[i] not in child:
            child[j] = parent2[i]
            j = j + 1 

In [243]:
def partially_mapped_crossover(parent1, parent2, child1, child2):
    cutoff_1, cutoff_2 = sorted(random.sample(range(len(parent1) + 1), 2))
    
    child1 = deepcopy(parent1)
    child2 = deepcopy(parent2)
    
    child1[cutoff_1:cutoff_2] = parent2[cutoff_1:cutoff_2]
    child2[cutoff_1:cutoff_2] = parent1[cutoff_1:cutoff_2]

   

    middle_map = {}
    for a, b in zip (child1[cutoff_1:cutoff_2], child2[cutoff_1:cutoff_2]):
        middle_map[a] = middle_map.get(a, 0) + 1
        middle_map[b] = middle_map.get(b, 0) + 1
        
    mapping_order = []
    for key, value in middle_map.items():
        if value != 2:
            mapping_order.append(key)
            
    def find_all_occurrences(lst, item):
        return [i for i, x in enumerate(lst) if x == item]    
          
    def replace_a_b(x_pos, y, child, cutoff_1, cutoff_2):
        for i in x_pos:
            if i not in range(cutoff_1, cutoff_2):
                child[i] = y
                break 
                   
        return child
        
    def replace_a_b_child(a, a_pos, b, b_pos, child, cutoff_1, cutoff_2):
        if len(a_pos) != 0:
             return replace_a_b(a_pos, b, child, cutoff_1, cutoff_2)
        else:
            return replace_a_b(b_pos, a, child, cutoff_1, cutoff_2)
        
    for i in range(0, len(mapping_order)-1, 2):
        a = mapping_order[i]
        b = mapping_order[i+1]

        a_child1_positions = find_all_occurrences(child1, a)
        a_child2_positions = find_all_occurrences(child2, a)
        
        
        b_child1_positions = find_all_occurrences(child1, b)
        b_child2_positions = find_all_occurrences(child2, b)
        
        child1 = replace_a_b_child(a, a_child1_positions, b, b_child1_positions, child1, cutoff_1, cutoff_2)
        child2 = replace_a_b_child(a, a_child2_positions, b, b_child2_positions, child2, cutoff_1, cutoff_2)

    return child1, child2

[1 | 2 3 4 | 5]  

[5 | 4 3 1 | 2]   

[1 | 4 3 1 | 5]

[5 | 2 3 4 | 2]

4-2, 3-3, 1-4

1: 1

2: 1

3: 2

4: 2

1 -> 2


[2 | 1 3 4 | 5]

[5 | 2 3 4 | 1]
n gradova 
0 depo 
1-n customeri 
m vozila
n + i, i indeks vozila 
rutq 11 ruta 12 ruta 0 0 0 0 0


In [244]:
def crossover(param, parent1, parent2, child1, child2):
    #order_crossover(parent1, parent2, child1)
    #order_crossover(parent2pass, parent1, child2)
    child1, child2 = param(parent1, parent2, child1, child2)

    return child1, child2
    

In [245]:
# child1 = []
# child2 = []
# child1, child2 = crossover(partially_mapped_crossover,x, y, child1, child2)
# print(x.solution)


#Best route better adjustment

In [246]:
# parent1 = [1, 2, 3, 4, 5]
# parent2 = [5, 4, 3, 1, 2]
# child1 = [21, 22, 33, 44, 55]
# child2 = [11, 12, 13, 14, 15]

# crossover(parent1, parent2, child1, child2)

In [247]:
def swap_mutation(individual):
    l, r = sorted(random.sample(range(len(individual)), 2))
    individual[l], individual[r] = individual[r], individual[l]
    print(individual)

In [248]:
def invert_mutation(individual):
    l, r = sorted(random.sample(range(len(individual) + 1), 2))
    chosen = individual[l:r]
    chosen.reverse()
    individual[l:r] = chosen
   # print(individual)

In [249]:
def shaking_mutation(individual):
    l, r = sorted(random.sample(range(len(individual) + 1), 2))
    chosen = individual[l:r]
    random.shuffle(chosen)
    individual[l:r] = chosen
    print(individual)

In [250]:
child = [1, 2, 3, 4, 5]
swap_mutation(child)

child = [1, 2, 3, 4, 5]
invert_mutation(child)

child = [1, 2, 3, 4, 5]
shaking_mutation(child)

[2, 1, 3, 4, 5]
[1, 3, 2, 4, 5]


In [251]:
def mutation(param, individual, mutation_prob):
    if random.random() < mutation_prob:
        param(individual)

In [254]:
def genetic_algorithm(data, population_size, elitism_size, mutation_prob, tournament_size, num_generations, capacity,crossover_param, mutation_param, num_of_vehicles, service_time):
    population = [Individual(data, capacity, num_of_vehicles, len(data)-1, service_time ) for _ in range(population_size)]
    new_population = deepcopy(population)

    for i in range(num_generations):
        population.sort(key = lambda x: x.fitness)
        new_population[:elitism_size] = population[:elitism_size]
        for j in range(elitism_size, population_size, 2):
            parent1 = tournament_selection( population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            while(parent1 == parent2):
                parent2 = tournament_selection(population, tournament_size)
            
            crossover(crossover_param, parent1.solution, parent2.solution, child1= new_population[j].solution, child2 = new_population[j+1].solution)
            
            mutation(mutation_param, new_population[j].solution, mutation_prob)
            mutation(mutation_param, new_population[j+1].solution, mutation_prob)

            new_population[j].fitness = new_population[j].calc_fitness()
            new_population[j+1].fitness = new_population[j+1].calc_fitness()

        population = deepcopy(new_population)
    
    return min(population, key = lambda x: x.fitness)

In [257]:
POPULATION_SIZE = 1000
ELITISIM_SIZE = 100
MUTATION_PROB = 0.1
TOURNAMENT_SIZE = 300
NUM_GENERATIONS = 100
CAPACITY = 200
CROSSOVER = partially_mapped_crossover
MUTATION = invert_mutation
NUM_OF_VEHICLES = 19
SERVICE_TIME = 10

ga = genetic_algorithm(cities, POPULATION_SIZE, ELITISIM_SIZE, MUTATION_PROB, TOURNAMENT_SIZE, NUM_GENERATIONS, CAPACITY, CROSSOVER, MUTATION, NUM_OF_VEHICLES, SERVICE_TIME )

print(ga.solution)
print(ga.fitness)

[72, 35, 101, 67, 25, 102, 36, 58, 103, 95, 43, 104, 12, 55, 105, 79, 1, 106, 5, 7, 17, 107, 99, 97, 108, 100, 109, 4, 77, 110, 42, 85, 13, 111, 48, 112, 31, 3, 113, 89, 114, 28, 18, 93, 115, 2, 53, 70, 116, 88, 26, 117, 98, 61, 60, 118, 15, 6]
2055.106911134009


egzaktan, grafovi, viseciljna, reference, prezentacija i pdf

